# Masterthesis
#### Julian Jetz

In [1]:
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
from scipy import stats
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from pandas.plotting import scatter_matrix
from currency_converter import CurrencyConverter
from datetime import date


In [2]:
na_values = ['nan', 'N/A', 'NaN', 'NaT']

In [3]:
def get_merged_csv(flist, **kwargs):
    return pd.concat([pd.read_csv(f, **kwargs) for f in flist], ignore_index=True)

In [4]:
df = get_merged_csv(glob.glob('csvfiles/*.csv'), na_values=na_values, index_col=False, dtype = {"STATE" : "str", "TYPE" : "str", "TYPEQ" : "str", "WEATHER" : "str", "VISIBLTY" : "str"})
df.shape[0]

/Users/julianjetz/anaconda3/envs/masterthesis/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (119) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/julianjetz/anaconda3/envs/masterthesis/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (119,128,129,130,131,132,133,134,135) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/julianjetz/anaconda3/envs/masterthesis/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (7,27,41,119,125,126,127,128,129,130,131,132,133,134,135) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


62846

In [5]:
acc_causes = pd.read_csv('resources/ACC_CAUSE_LIST.csv',sep=';',encoding='latin1',index_col='ACCAUSE')

In [6]:
#track_speed_limits = pd.read_csv('resources/TRACK_SPEED_LIMIT.csv',sep=';',encoding='latin1',index_col='Track Classification')

In [7]:
#def getTooFast(trnspd, trkcls):

In [8]:
def getAccCauseLvl1(accause):
    return acc_causes.loc[accause]['ACCAUSE_LVL1']

In [9]:
def getAccCauseLvl2(accause):
    return acc_causes.loc[accause]['ACCAUSE_LVL2']

In [10]:
df['ACCCAUSE_LVL1']=df.apply(lambda row : getAccCauseLvl1(row['ACCAUSE']),axis=1)

KeyboardInterrupt: 

In [ ]:
df['ACCCAUSE_LVL2']=df.apply(lambda row : getAccCauseLvl2(row['ACCAUSE']),axis=1)

In [ ]:
df['ACCCAUSE_LVL1'].unique()

In [ ]:
df['LOCOMOTIVES1']=df.apply(lambda row : row['HEADEND1']+row['MIDMAN1']+row['MIDREM1']+row['RMAN1']+row['RREM1'],axis=1)

In [ ]:
df['LOCOMOTIVES2']=df.apply(lambda row : row['HEADEND2']+row['MIDMAN2']+row['MIDREM2']+row['RMAN2']+row['RREM2'],axis=1)

In [ ]:
def getEuro(damage, year, month, day):
    c = CurrencyConverter(fallback_on_missing_rate=True)
    damage=c.convert(damage, 'USD', 'EUR', date=date(year, month, day))
    return damage

#### Schaden in Euro umwandeln

In [ ]:
#df['ACCDMG'] = df.apply(lambda row : getEuro(row['ACCDMG'], row['YEAR4'], row['MONTH'], row['DAY']), axis=1)

In [ ]:
def isSerious(accdmg, killed, injured):
    type="Not Significant"
    # killed=row['RREMPKLD']+row['PASSKLD']+row['OTHERKLD']
    # injured=row['RREMPINJ']+row['PASSINJ']+row['OTHERINJ']
    if accdmg>=150000 or injured > 0 or killed > 0:
        type="Significant"
    return type

#### Bestimmung der Schwere des Unfalls (Nicht Signifikant, Signifikant und Schwer)

In [ ]:
df['ACCTYPE'] = df.apply(lambda row : isSerious(row['ACCDMG'],row['RREMPKLD'] + row['PASSKLD'] + row['OTHERKLD'],row['RREMPINJ'] + row['PASSINJ'] + row['OTHERINJ']), axis=1)

*Umwandlung Monat zu Jahreszeiten*
*TODO*

In [ ]:
pd.set_option('display.max_columns', 500)
#df.head(50)

In [ ]:
#imp = IterativeImputer(missing_values=np.nan, sample_posterior=False, max_iter=10, tol=0.001, n_nearest_features=4, initial_strategy='median')
#imp.fit(df)
#imputed_df = pd.DataFrame(df=imp.transform(df), columns=['TYPTRK'], dtype='int')

In [ ]:
df[df['TYPEQ'].isnull()].shape[0]

In [ ]:
df[df['TYPTRK'].isnull()].shape[0]

In [ ]:
df[df['TRNSPD']==0].shape[0]

In [ ]:
df[df['TONS']==0].shape[0]

#### Entfernung aller n/a Werte

In [ ]:
df=df.dropna(subset=['TYPEQ', 'TYPTRK']);

df.shape[0]


In [ ]:
df.head(50)

In [ ]:
duplicateDF = pd.concat(g for _, g in df.groupby("INCDTNO") if len(g) > 1).copy()
duplicateDF.sort_values(by=['INCDTNO'], inplace=True)

In [ ]:
duplicateDF

In [ ]:
std_dev = 3
df[(np.abs(stats.zscore(df[['LOCOMOTIVES1','LOADF1', 'LOADP1', 'EMPTYF1', 'EMPTYP1','LOCOMOTIVES2','LOADF2', 'EMPTYF2', 'TONS', 'TEMP', 'ACCDMG']])) >= float(std_dev)).all(axis=1)]

In [ ]:
df.drop(df.loc[df['ACCTYPE']=="Not Significant"].index, inplace=True)
df.shape[0]

#### Ausreißer entfernen

In [ ]:
std_dev = 3
df = df[(np.abs(stats.zscore(df[['LOCOMOTIVES1','LOADF1', 'LOADP1', 'EMPTYF1', 'EMPTYP1','LOCOMOTIVES2','LOADF2', 'EMPTYF2', 'TONS', 'TEMP', 'ACCDMG']])) < float(std_dev)).all(axis=1)]
df.shape[0]

##### TODO: Prüfen Adaptive Learning rate

#### [Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam) <br>
Die Adam-Optimierung ist eine stochastische Gradientenabsenkungsmethode, die auf einer adaptiven Schätzung von Momenten erster und zweiter Ordnung basiert. Das Verfahren ist "recheneffizient, hat wenig Speicherbedarf, ist invariant gegenüber der diagonalen Neuskalierung von Gradienten und eignet sich gut für Probleme, die in Bezug auf Daten/Parameter groß sind".
[Arxiv](https://arxiv.org/pdf/1412.6980.pdf)<br><br>
*Learning rate:* In der maschinellen Lern- und Statistiktechnik ist die Lernrate ein Tuningparameter in einem Optimierungsalgorithmus, der die Schrittweite bei jeder Iteration bestimmt und sich dabei auf ein Minimum einer Verlustfunktion zubewegt. Da sie beeinflusst, inwieweit neu gewonnene Informationen alte Informationen übersteuern, stellt sie metaphorisch die Geschwindigkeit dar, mit der ein maschinelles Lernmodell "lernt". Bei der Festlegung einer Lernrate gibt es einen Kompromiss zwischen der Konvergenzrate und der Überschreitung. Während die Richtung zum Minimum in der Regel aus dem Gradienten der Verlustfunktion bestimmt wird, bestimmt die Lernrate, wie groß ein Schritt in diese Richtung ist.Eine zu hohe Lernrate führt dazu, dass der Lernsprung über Minima hinausgeht, aber eine zu niedrige Lernrate dauert entweder zu lange, um sich zu konvergieren oder in einem unerwünschten lokalen Minimum stecken zu bleiben.


In [ ]:
opti = tf.optimizers.Adam(learning_rate = 0.001)

#### [Nadam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Nadam?authuser=2&hl=vi&version=stable)
[Incorporating Nesterov Momentum into Adam](http://cs229.stanford.edu/proj2015/054_report.pdf)

In [ ]:
# opti = tf.optimizers.Nadam(learning_rate = 0.01)

#### [Adagrad Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adagrad?hl=vi&authuser=2&version=stable)
Adagrad ist ein Optimierer mit parameter-spezifischen Lernraten, die angepasst werden, je nachdem, wie häufig ein Parameter während des Trainings aktualisiert wird. Je mehr Updates ein Parameter erhält, desto kleiner sind die Updates. [Adaptive Subgradient Methods for Online Learning and Stochastic Optimization](http://www.jmlr.org/papers/volume12/duchi11a/duchi11a.pdf)

In [ ]:
#opti = tf.optimizers.Adagrad(learning_rate = 0.001)

#### [Adamax Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adamax?hl=vi&authuser=2&version=stable)
Adamax ist eine Variante von Adam, die auf der Unendlichkeitsnorm basiert. Adamax ist Adam manchmal überlegen, besonders bei Modellen mit Einbettungen. [Arxiv](https://arxiv.org/pdf/1412.6980.pdf)

In [ ]:
# opti = tf.optimizers.Adamax(learning_rate = 0.01)

## Complete regression model

Festlegen der abhängigen Variablen X und der vorherzusagenden Variable y

In [ ]:
x_data = df[['YEAR4', 'MONTH', 'LOCOMOTIVES1', 'LOADF1', 'LOADP1', 'EMPTYF1', 'EMPTYP1','LOCOMOTIVES2', 'LOADF2', 'EMPTYF2', 'TONS', 'TEMP', 'SPEED','TRNSPD', 'ACCCAUSE_LVL1', 'TYPE', 'TYPEQ', 'TYPTRK', 'VISIBLTY', 'WEATHER', 'STATE']]
y = df['ACCDMG']

In [ ]:
#x_data = df[df['ACCTYPE']=="Significant"][['YEAR4', 'MONTH', 'LOADF1', 'LOADP1', 'EMPTYF1', 'EMPTYP1','LOADF2',  'EMPTYF2', 'TONS', 'TEMP', 'TRNSPD', 'ACCAUSE', 'TYPE', 'TYPEQ', 'TYPTRK', 'VISIBLTY', 'WEATHER', 'COUNTY']].copy()
#y = df[df['ACCTYPE']=="Significant"][['ACCDMG']].copy()

In [ ]:
#cause_dummies= pd.get_dummies(df['ACCAUSE'], prefix='cause')
#x_data = pd.concat([x_data, df[['ACCAUSE', 'TYPE', 'TYPEQ', 'TYPTRK', 'VISIBLTY', 'WEATHER', 'COUNTY']], axis=1)

In [ ]:
x_data.describe(include='all')

In [ ]:
y.describe(include='all')

#### Scattermatrix und Korrelationen der Daten
Jahr, Monat, Anzahl der beladenen Frachtwagons und Personenwagons, Gewicht, Temperatur, Geschwindigkeit und Unfallkosten

In [ ]:
scatter_matrix(df[df['ACCTYPE']=="Significant"][['YEAR4', 'MONTH','LOCOMOTIVES1', 'LOADF1', 'LOADP1', 'EMPTYF1', 'EMPTYP1','LOCOMOTIVES2','LOADF2', 'EMPTYF2', 'TONS', 'TEMP', 'TRNSPD', 'ACCCAUSE_LVL1', 'TYPE', 'TYPEQ', 'TYPTRK', 'VISIBLTY', 'WEATHER', 'STATE', 'ACCDMG']], diagonal='kde', figsize=(18, 18))
plt.savefig(r"Images/scatterALL.png")

In [ ]:
colormap = plt.cm.RdBu
corr = df[df['ACCTYPE']=="Significant"][['ACCDMG', 'YEAR4', 'MONTH', 'LOCOMOTIVES1', 'LOADF1', 'LOADP1', 'EMPTYF1', 'EMPTYP1','LOCOMOTIVES2','LOADF2', 'EMPTYF2', 'TONS', 'TEMP', 'TRNSPD', 'ACCCAUSE_LVL1', 'TYPE', 'TYPEQ', 'TYPTRK', 'VISIBLTY', 'WEATHER', 'STATE']].corr()
corr

In [ ]:
ax=plt.figure(figsize=(20,15)).gca()
sns.heatmap(corr,ax=ax,linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True);
plt.savefig(r"Images/corrALL.png")

#### Aufteilen der Daten in Train und Test Datensatz

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y, test_size=0.3)

#### Festlegen der numerischen Merkmalsspalten

In [ ]:
trnspd = tf.feature_column.numeric_column('TRNSPD')
year = tf.feature_column.numeric_column('YEAR4')
month = tf.feature_column.numeric_column('MONTH')
locomotives1 = tf.feature_column.numeric_column('LOCOMOTIVES1')
loadf1 = tf.feature_column.numeric_column('LOADF1')
loadp1 = tf.feature_column.numeric_column('LOADP1')
emptyf1 = tf.feature_column.numeric_column('EMPTYF1')
emptyp1 = tf.feature_column.numeric_column('EMPTYP1')
locomotives2 = tf.feature_column.numeric_column('LOCOMOTIVES2')
loadf2 = tf.feature_column.numeric_column('LOADF2')
emptyf2 = tf.feature_column.numeric_column('EMPTYF2')
tons = tf.feature_column.numeric_column('TONS')
temp = tf.feature_column.numeric_column('TEMP')

In [ ]:
x_data[['ACCCAUSE_LVL1', 'TYPE', 'TYPEQ', 'TYPTRK', 'VISIBLTY', 'WEATHER', 'STATE']].astype(str).values

#### Festlegen der kategorischen Merkmalsspalten. 
Anstatt die Daten als einen one-hot Vektor mit vielen Dimensionen darzustellen, stellt eine Einbettungsspalte diese Daten als einen niederdimensionalen, dichten Vektor dar, in dem jede Zelle eine beliebige Zahl enthalten kann, nicht nur 0 oder 1. Die Größe der Einbettung ist ein Parameter, der angepasst werden muss (*TODO*).

In [ ]:
accause = tf.feature_column.categorical_column_with_hash_bucket('ACCCAUSE_LVL1',hash_bucket_size=10000)

embedding_size = int(math.floor(len(x_data['ACCCAUSE_LVL1'].unique())**0.25))
accause=tf.feature_column.embedding_column(accause, dimension=embedding_size)

In [ ]:
acctype = tf.feature_column.categorical_column_with_hash_bucket('TYPE',hash_bucket_size=10000)

embedding_size = int(math.floor(len(x_data['TYPE'].unique())**0.25))
acctype=tf.feature_column.embedding_column(acctype, dimension=embedding_size)

In [ ]:
typeq = tf.feature_column.categorical_column_with_hash_bucket('TYPEQ',hash_bucket_size=10000)

embedding_size = int(math.floor(len(x_data['TYPEQ'].unique())**0.25))
typeq=tf.feature_column.embedding_column(typeq, dimension=embedding_size)

In [ ]:
typtrk = tf.feature_column.categorical_column_with_hash_bucket('TYPTRK',hash_bucket_size=10000)

embedding_size = int(math.floor(len(x_data['TYPTRK'].unique())**0.25))
typtrk=tf.feature_column.embedding_column(typtrk, dimension=embedding_size)

In [ ]:
visibility = tf.feature_column.categorical_column_with_hash_bucket('VISIBLTY',hash_bucket_size=10000)

embedding_size = int(math.floor(len(x_data['VISIBLTY'].unique())**0.25))
visibility=tf.feature_column.embedding_column(visibility, dimension=embedding_size)

In [ ]:
weather = tf.feature_column.categorical_column_with_hash_bucket('WEATHER',hash_bucket_size=10000)

embedding_size = int(math.floor(len(x_data['WEATHER'].unique())**0.25))
weather=tf.feature_column.embedding_column(weather, dimension=embedding_size)

In [ ]:
state = tf.feature_column.categorical_column_with_hash_bucket('STATE',hash_bucket_size=10000)

embedding_size = int(math.floor(len(x_data['STATE'].unique())**0.25))
state=tf.feature_column.embedding_column(state, dimension=embedding_size)

#### Festlegen der Merkmalsspalten

In [ ]:
feature_col =[year, month, trnspd, typeq, locomotives1, loadf1, loadp1, emptyf1, emptyp1, locomotives2, loadf2, emptyf2, typtrk, acctype, accause, visibility, weather, temp, state]

#### Aufstellen der Input Funktion

Batch_Size=Größe der zurückzusendenden Batches.<br>
Num_Epochs=Anzahl der Perioden, die man über Daten iterieren muss.<br>
Shuffle=Sollendie Datensätze in zufälliger Reihenfolge gelesen werden?

In [ ]:
input_func= tf.compat.v1.estimator.inputs.pandas_input_fn(x=x_train, 
                                                y= y_train, 
                                                batch_size=10, 
                                                num_epochs=1000, 
                                                 shuffle=True)

#### Aufstellen der Eval Input Funktion

In [ ]:
test_input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x=x_test,                                                   
                                                 batch_size=10, 
                                                 num_epochs=1, 
                                                 shuffle=False)
eval_input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x=x_test,
                                                      y=y_test, 
                                                      batch_size=10, 
                                                      num_epochs=1, 
                                                      shuffle=False)
train_input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x=x_train,                                                   
                                                 batch_size=10, 
                                                 num_epochs=1, 
                                                 shuffle=False)

#### Initialisierung des Estimators (DNNRegressor)
hidden_units=Das Argument hidden_units ermöglicht es, ein Array mit der Anzahl der Knoten für jede Schicht zu erzeugen. Dies ermöglicht es, ein neuronales Netzwerk zu erstellen, indem einfach seine Größe und Form berücksichtigt wird, anstatt das Ganze von Grund auf von Hand zu vernetzen. (TODO: fine tune)

In [ ]:
estimator = tf.estimator.DNNRegressor(hidden_units=[15,30,45,30], feature_columns=feature_col, optimizer=opti, dropout=0.5)

#### Train Model

In [ ]:
estimator.train(input_fn=input_func, max_steps=60000)


#### Evaluate Model mit Eval input function

In [ ]:
result_eval = estimator.evaluate(input_fn=eval_input_func)
result_eval

#### Scatterplot Vergleich tatsächliche und vorhergesagte Werte

In [ ]:
predictions=[]
for pred in estimator.predict(input_fn=test_input_func):
    predictions.append(pred['predictions'][0].astype(float))
plt.plot(y_test, predictions, 'o')
plt.xlabel('Actual values (test data)')
plt.ylabel('predicted values (test data)')

In [ ]:
train_predictions=[]
for pred in estimator.predict(input_fn=train_input_func):
    train_predictions.append(pred['predictions'][0].astype(float))
plt.plot(y_train, train_predictions, 'o')
plt.xlabel('Actual values (train data)')
plt.ylabel('predicted values (train data)')

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, predictions))**0.5
rmse

#### Verleich tatsächliche und vorhergesagte Werte 
Beispiel: 30 zufällig ausgewählte Werte

In [ ]:
pred = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
pred1 = pred.sample(100)

pred1.plot(kind='bar',figsize=(20,16))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

#### Residual Plot 
Abweichung zwischen den vorhergesagten und tatsächlichen Unfallkosten (Testdaten=grün, Trainingsdaten=blau)

In [ ]:
    plt.scatter(train_predictions, train_predictions - y_train, c='b', s=40, alpha=0.5, label='Train Data')
    plt.scatter(predictions, predictions - y_test, c='g', s=40, label='Test Data')
    plt.hlines(y=0, xmin=-0.03, xmax=0.2)
    plt.title('Residual Plot of DNN Regression')
    plt.ylabel('Residuals')
    plt.xlabel('Accident Damage')
    plt.legend()
    plt.show()